In [79]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from sklearn.utils import shuffle

import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.data import DataLoader

from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import add_self_loops, degree

from src.data_utils import load_dataset

# Scope of the notebook

In this notebook, we will use a Graph Convolutional Neural Network to solve the ESOL regression task.

At the beginning you need to take the molecular graph data and transform it to a valid format in order to fit the data to PyTorch geometric layers.

Then, you will have to implement a "vanilla" graph convolutional layer, define the network module that uses this layer and train it, using predefined hyperparameters.

At the end, you will have to make a random search of hyperparameters in order to obtain the best possible prediction score.

As the training loss and scoring function we will use mean squared error (MSE).

# Load the dataset

In [156]:
path = './data/'
target_name = 'ESOL'
batch_size = 32
task = 'regression'

train_dataset = load_dataset(path, target_name, 'train')
valid_dataset = load_dataset(path, target_name, 'val')
test_dataset = load_dataset(path, target_name, 'test')

filted_atomtype_list_order: [6, 8, 7, 17, 16, 9, 35, 15, 53, 'Others'], 
 filted_bondtype_list_order: ['6_6', '6_8', '6_7', '6_17', '6_16', '7_8', '8_15', '6_9', '15_16', '6_35', '8_16', '7_7', 'Others']
Transfer mol to matrices
Done.
filted_atomtype_list_order: [6, 8, 7, 17, 9, 35, 'Others'], 
 filted_bondtype_list_order: ['6_6', '6_8', '6_7', '6_17', 'Others']
Transfer mol to matrices
Done.
filted_atomtype_list_order: [6, 8, 7, 17, 9, 16, 35, 'Others'], 
 filted_bondtype_list_order: ['6_6', '6_8', '6_7', '6_17', 'Others']
Transfer mol to matrices
Done.


# Create data loaders for pytorch geometric

Every molecule in our dataset is now represented by a 3-tuple which consists of:
 - Adjacency matrix $\in \mathbb{M}$(n_atoms, n_atoms)
 - Atom features matrix $\in \mathbb{M}$(n_atoms, n_features)
 - Label (ESOL value)
 
[PyTorch Geometric requires a different form of the data](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#module-torch_geometric.data). 

Please write the function that transforms the data into the correct format:
 - Atom features matrix $\in \mathbb{M}$(n_atoms, n_features) - denoted by *x*
 - Label (ESOL value) - denoted by *y*
 - Edge indices matrix $\in \mathbb{M}$(2, n_edges) - denoted by *edge_index*

In [164]:
def get_edge_indices(adj):
    edges_list = []
    for i in range(adj.shape[0]):
        for j in range(i, adj.shape[0]):
            if adj[i,j] == 1:
                edges_list.append((i,j))
    return edges_list


def transform_molecule_pg(mol):
    adj = mol[0]
    afm = mol[1]
    label = mol[2]
        
    x = torch.tensor(afm)
    y = torch.tensor(label)
    edge_index = torch.tensor(get_edge_indices(adj)).t().contiguous()
    
    return Data(x=x, y=y, edge_index=edge_index)


def transform_dataset_pg(dataset):
    dataset_pg = []
    
    for mol in dataset:
        dataset_pg.append(transform_molecule_pg(mol))
        
    return dataset_pg

In [165]:
train_dataset = transform_dataset_pg(train_dataset)
valid_dataset = transform_dataset_pg(valid_dataset)
test_dataset = transform_dataset_pg(test_dataset)

In [166]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the vanilla GCN layer

Please write a vanilla graph convolutional layer that inherits from [MessagePassing layer](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.message_passing.MessagePassing).

For every atom it should take all its neighbours and then:
1. Apply a linear layer on their feature vectors
2. Apply a ReLU nonlinearity
3. Aggregate information by taking the mean value of the feature vectors of all neighbours.

i.e. $x^{k}_{i} = \frac{1}{|N(i)|} \sum_{j \in N(i)} \text{ReLU}(Wx_{j} + b)$, where $N(i) = \{ j\ |\ (i, j) \in E \}$

Reminder: Make sure that self loops are included in the adjacency matrix.

In [176]:
class Vanilla_GC_Layer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(Vanilla_GC_Layer, self).__init__(aggr='mean')  # "Mean" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        
        # Step 3: Add non-linearity
        x = F.relu(x)
        
        # Step 4: Start propagating messages.
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        # x_j has shape [E, out_channels]
        return x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]
        return aggr_out

# Create and train GC

Please define a class for the Graph Convolutional Network that uses our predefined `Vanilla_GC_Layer`. For the input graph, network should:
1. Pass it through some number of GC layers
2. Aggregate information from the whole molecule, by applying global mean pooling
3. Pass the graph embedding into the linear layer that returns the predicted value

In the class definition you should include the following network hyperparameters:
 - *layers_num* - number of Graph Convolutional layers
 - *model_dim* - dimensionality of the model inner representation
 - *input_dim* - dimensionality of the input atom representation
 - *output_dim*  - dimensionality of the output vector

In [177]:
class GraphConvNetwork(torch.nn.Module):
    def __init__(self, layers_num, model_dim, 
                 input_dim, output_dim):
        super(GraphConvNetwork, self).__init__()
        self.layers_num = layers_num
        
        self.conv_layers = [Vanilla_GC_Layer(in_channels=input_dim, 
                                             out_channels=model_dim)] + \
                            [Vanilla_GC_Layer(in_channels=model_dim, 
                                              out_channels=model_dim) 
                            for _ in range(layers_num-1)]
        
        self.conv_layers = torch.nn.ModuleList(self.conv_layers)
        self.out_layer = torch.nn.Linear(model_dim, output_dim)

    def forward(self, data):
        for i in range(self.layers_num):
            data.x = self.conv_layers[i](data.x, data.edge_index)
            
        data.x = global_mean_pool(data.x, data.batch)
        x = self.out_layer(data.x)

        return x

Define the following functions for training and validating our network:

1. Function **train** that trains the model for a given number of epochs. The function takes the *model* and *optimizer* as parameters. For every epoch step it runs the *run_epoch* function and then it calculates the MSE loss for the valid data.  

2. Function **run_epoch** that trains the model for a single epoch step.  The function takes the *model*, *optimizer* and *data_loader* as parameters. It should return the train loss for the given epoch.

3. Function **valid** that calculates the scoring function - mean squared error for the given dataset. The function takes the *model* and *data_loader* as parameters. It should return the calculated MSE score.

With such functions definitions, parts of the code could be re-used in the next sections of the notebook.

In [178]:
def train(model, optimizer):
    for epoch in range(epochs_num):
        epoch_train_loss = run_epoch(model, optimizer, train_loader)
        epoch_valid_loss = valid(model, valid_loader)
        print(f'Epoch: {epoch}, train loss: {epoch_train_loss}, valid loss: {epoch_valid_loss}')
    
    test_loss = valid(model, test_loader)
    print(f'End of training, test loss: {test_loss}')

        
def run_epoch(model, optimizer, data_loader):
    model.train()
    
    loss_all = 0
    for data in data_loader:
        data = data.to(device)
        optimizer.zero_grad()
        
        output = model(data).view(-1)
        loss = F.mse_loss(output, data.y)
        loss.backward()
        
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        
    return loss_all / len(train_dataset)


def valid(model, data_loader):
    model.eval()

    pred_array = []
    true_array = []
    
    for data in data_loader:
        data = data.to(device)
        output = model(data).view(-1)

        pred_array.append(output.detach().cpu().numpy())
        true_array.append(data.y.cpu().numpy())
        
    pred_array = np.concatenate(pred_array)
    true_array = np.concatenate(true_array)
    
    return mean_squared_error(true_array, pred_array)

In [179]:
# Set some default network hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layers_num = 3
model_dim = 32
input_dim = train_dataset[0].x.shape[1]
output_dim = train_dataset[0].y.shape[0]

lr = 0.0001
epochs_num = 100

In [180]:
# Define the model and the optimizer
model = GraphConvNetwork(
            layers_num=layers_num, 
            model_dim=model_dim, 
            input_dim=input_dim, 
            output_dim=output_dim).to(device)
                   
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [181]:
# Train the network!
train(model, optimizer)

Epoch: 0, train loss: 1.0175402674600977, valid loss: 0.8668918609619141
Epoch: 1, train loss: 1.0146582222764613, valid loss: 0.8658786416053772
Epoch: 2, train loss: 1.0121855645909277, valid loss: 0.865025520324707
Epoch: 3, train loss: 1.0099143188141138, valid loss: 0.8645021319389343
Epoch: 4, train loss: 1.0077148969316165, valid loss: 0.8636859655380249
Epoch: 5, train loss: 1.0057702837656448, valid loss: 0.8626046776771545
Epoch: 6, train loss: 1.004155245694247, valid loss: 0.8615769743919373
Epoch: 7, train loss: 1.0021117693040429, valid loss: 0.8602099418640137
Epoch: 8, train loss: 1.0001901101378803, valid loss: 0.8591213822364807
Epoch: 9, train loss: 0.9983266911855558, valid loss: 0.8576339483261108
Epoch: 10, train loss: 0.9962621632542156, valid loss: 0.8560516834259033
Epoch: 11, train loss: 0.9941150420520892, valid loss: 0.8541402816772461
Epoch: 12, train loss: 0.9918689990783742, valid loss: 0.8519949316978455
Epoch: 13, train loss: 0.9895103635914838, valid l

# Hyperparameter search to obtain better results

In [182]:
# The function that takes the given number of random samples 
# from a given hyperparameters distributions.
def make_params_grid(params, max_parameter_sets, randomize=True):
    to_list = lambda x: [x] if not isinstance(x, Iterable) else x
    params = {k: to_list(v) for k, v in params.items()}
    if randomize:
        grid = shuffle(ParameterGrid(params))
        return grid[:max_parameter_sets]
    return ParameterGrid(params)

In our experiment we will look for the best setting of the following hyperparameters:
 - learning rate
 - epochs_num
 - batch_size
 - layers_num
 - model_dim


You should define a function that searches for the best set of hyperparameters from a predefined distribution. 

1. The function **train_for_params** creates and trains the model for a single hyperparameters setting. The function takes *params* as parameters. It should:
    1. Define the data loaders (as they need the *batch_size* parameter that we just sampled).
    2. Define the network (as it needs the *layers_num* and *model_dim* parameters that we just sampled).
    3. Define the optimizer (as it needs the *learning_rate* parameter that we just sampled).
    4. Train the network with a given optimizer and data loaders, for a given number of epochs (*epochs_num* parameter that we just sampled) - here you can reuse the *run_epoch* function.
    5. Test the trained model on valid data (you can also calculate the MSE on test data there, to make your work easier).
    6. Return the valid and test loss for the given hyperparameters setting.

2. The function **grid_search** looks for the best hyperparameters setting for a given number of trials. The function takes *param_grid* and *max_parameter_sets* as arguments. For a single step, it should take the sampled hyperparameters, call the function *train_for_params* and save the validation score of the resulted model. At the end, it should take the best model selected by finding the lowest validation score and return the test score for the chosen model.

In [183]:
def train_for_params(params):
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=params['batch_size'])
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'])
    
    model = GraphConvNetwork(
                    layers_num=params['layers_num'], 
                    model_dim=params['model_dim'], 
                    input_dim=input_dim,
                    output_dim=output_dim).to(device)
                   
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])
    
    for epoch in range(params['epochs_num']):
        epoch_train_loss = run_epoch(model, optimizer, train_loader)
    
    model_valid_loss = valid(model, valid_loader)
    model_test_loss = valid(model, test_loader)
    return model_valid_loss, model_test_loss

def grid_search(param_grid, max_parameter_sets):
    best_params = {}
    best_val_score = np.inf
    best_test_score = np.inf
    
    param_grid = make_params_grid(param_grid, max_parameter_sets, randomize=True)
    
    for i, params in enumerate(param_grid):
        model_valid_loss, model_test_loss = train_for_params(params)
        print(f'Model: {i} trained, valid loss: {model_valid_loss}')
        
        if model_valid_loss < best_val_score:
            best_params = params
            best_val_score = model_valid_loss
            best_test_score = model_test_loss
            
    print(f'Best model valid loss: {best_val_score}, test loss: {best_test_score}')
    print(f'Best model hyperparams: {best_params}')

In [184]:
# Define the params grid for our random search and set the maximum number of trials
param_grid = {
                'lr': [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001],
                'epochs_num': [20, 50, 100, 200],
                'batch_size': [16, 32, 64, 128],
                'layers_num': [1, 2, 4, 6, 8],
                'model_dim': [16, 32, 64, 128, 256, 512],
             }

max_parameter_sets = 50

In [185]:
# Search for the best model!
grid_search(param_grid=param_grid, max_parameter_sets=50)

Model: 0 trained, valid loss: 0.40312573313713074
Model: 1 trained, valid loss: 0.46857258677482605
Model: 2 trained, valid loss: 0.8175060153007507
Model: 3 trained, valid loss: 0.31216463446617126
Model: 4 trained, valid loss: 0.3382140100002289
Model: 5 trained, valid loss: 0.8734336495399475
Model: 6 trained, valid loss: 0.3338407874107361
Model: 7 trained, valid loss: 0.4891279339790344
Model: 8 trained, valid loss: 0.434187114238739
Model: 9 trained, valid loss: 0.6572940945625305
Model: 10 trained, valid loss: 0.533747136592865
Model: 11 trained, valid loss: 0.33549633622169495
Model: 12 trained, valid loss: 0.39635682106018066
Model: 13 trained, valid loss: 0.859822154045105
Model: 14 trained, valid loss: 0.8818231225013733
Model: 15 trained, valid loss: 0.4685022532939911
Model: 16 trained, valid loss: 0.88692706823349
Model: 17 trained, valid loss: 0.421007364988327
Model: 18 trained, valid loss: 0.6426513195037842
Model: 19 trained, valid loss: 0.32032981514930725
Model: 20 

# Test other network settings

In order to to obtain the best possible score, you could experiment with other network settings, such as:
 - Adding dropout
 - Adding residual connections
 - Using different types of graph convolutional layer
 - Using different aggregation types
 - Adding more layer after the aggregation
 - Extending the grid, that we used for hyperparameters search